In [ ]:
import numpy as np
import gurobipy as grb
import csv
import time
import statsmodels.api as sm
from joblib import Parallel,delayed

def five_variate(n,SNR,rho):
    beta1 = np.array([1,1,1,0,1])
    beta2 = np.array([-1,-1,-1,0,-1])
    p = len(beta1)
    Sigma = np.zeros((p,p));
    for i in range(p):
        for j in range(p):
            Sigma[i,j] = rho**(abs(i-j))
    X = np.random.multivariate_normal(mean =np.zeros(p), cov = Sigma, size = n)
    noise1 = np.random.normal(0,np.sqrt(np.var(X[:int(n/2)]@beta1)/SNR), size = int(n/2)) 
    noise2 = np.random.normal(0,np.sqrt(np.var(X[:int(n/2)]@beta2)/SNR), size = int(n/2)) 
    Y1 = X[:int(n/2)]@beta1 + noise1
    Y2 = X[int(n/2):]@beta2 + noise2
    Y = np.r_[Y1,Y2]
    lam = 0.1*n
    #ols = sm.OLS(Y, X)
    #ols_result = ols.fit()
    #bigM = 10*float(1.96*(sum(ols_result.bse)))#1000

    M = grb.Model()
    M.Params.TimeLimit = 400
    beta = M.addMVar((int(n),5), lb = -grb.GRB.INFINITY)
    z = M.addMVar(len(Y) - 1, vtype = grb.GRB.BINARY)
    sum_of_sq = sum((Y[t] - sum(X[t,j]*beta[t,j] for j in range(5)))* (Y[t] - sum(X[t,j]*beta[t,j] for j in range(5))) for t in range(len(Y)))
    for t in range(len(Y)-1):
        for j in range(5):
            #M.addConstr(beta[t+1,j] - beta[t,j] <= bigM*z[t])
            #M.addConstr(beta[t+1,j] - beta[t,j] >= -bigM*z[t])
            M.addConstrs((z[t] == 0) >> (beta[t+1, j] == beta[t, j])) # not using big M
        
    #optional constratins to add more cuts
    M.addConstr(sum(z) <= 5)
    for t in range(1, len(Y) - 2):
        M.addConstr(z[t-1] + z[t] + z[t+1] <= 1)
    M.setObjective(sum_of_sq+ lam*sum(z))
    M.optimize()
    beta_hat = [beta[t].x for t in range(len(Y))]
    z_hat = [z[t].x for t in range(len(Y)-1)]
    opt_gap = M.MIPGap
    return z_hat, opt_gap,beta_hat

SNR_list = np.array([6, 3.52, 2.07, 1.22, 0.71])
rho_list = np.array([0, 0.3, 0.7])
n_list = np.array([1000])

header = ['Repitition','Time','rho', 'SNR', 'n', 'Optimality Gap', 'Change Points']#, 'beta_hat', 'nonzero count']
f = open('simulation_five_variate.csv', 'w')
writer = csv.writer(f)
writer.writerow(header)
f.close()

def to_repeat(rep):
    for n in n_list:
        for SNR in SNR_list:
            for rho in rho_list:
                tik = time.time()
                z_hat, opt_gap, beta_hat = five_variate(n,SNR,rho)
                tok = time.time()
                duration = tok - tik
                cps = [idx for idx,item in enumerate(z_hat) if item == 1 ]
                # Write result
                resultrow = [rep,duration,rho,SNR, n, opt_gap, cps]
                f = open('simulation_five_variate.csv', 'a')
                writer = csv.writer(f)
                writer.writerow(resultrow)
                f.close()
                
Parallel(n_jobs = 8)(delayed(to_repeat)(rep) for rep in range(500)) #Tomo's machine has 8 cores.